In [ ]:
import pandas
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

import pandas as pd
import geopandas as gpd

from bokeh.io import export_png, show
from bokeh.models import GeoJSONDataSource

from plotting import heatmap
from utils import timer

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
# Run in project dir
cur_dir = Path(os.path.abspath(''))
if cur_dir.name == 'notebooks':
    os.chdir(cur_dir.parent)
print(os.getcwd())

In [ ]:
# Smaller test range:
# start = '2020-08-01'
# end = '2020-08-30'

# Full date range
start = '2020-07-01'
end = '2023-06-30'

In [ ]:
# Load data
with timer('Load data'):
    whales_interp = gpd.read_parquet(f'data/intermediate/whale_pts_final_{start}_{end}.parquet')
    vessel_points = gpd.read_parquet(f'data/intermediate/vessel_pts_final_{start}_{end}.parquet')
    protected_areas = gpd.read_parquet('data/intermediate/protected_final.parquet')
    whales_segs = gpd.read_parquet(f'data/intermediate/whale_seg_final_{start}_{end}.parquet')
    vessel_segs = gpd.read_parquet(f'data/intermediate/vessel_seg_final_{start}_{end}.parquet')
    basemap = gpd.read_parquet('data/intermediate/basemap_final.parquet')

use_encounters = True

if use_encounters:
    vessel_encounters = gpd.read_parquet(f'data/intermediate/vessel_encounters_final_{start}_{end}.parquet')
    # whale_encounters = gpd.read_parquet(f'data/intermediate/whale_encounters_final_{start}_{end}.parquet')
else:
    vessel_encounters = None

In [ ]:
bounds = 'auck'
timestamps = pd.date_range(start, end, freq='30min')

In [ ]:

basemap_src = GeoJSONDataSource(geojson=basemap.to_json(default=str))

# Map bounds
bounds_full = vessel_points.geometry.total_bounds
bounds_ant = [178, -50.5, 179.5, -47]
bounds_auck = [165.25, -51.5, 167.25, -49.5]
bounds_camp = [168.25, -53, 169.75, -52]

bds = {
    'full': bounds_full,
    'auck': bounds_auck,
    'camp': bounds_camp,
    'anti': bounds_ant,
}[bounds]

done = 0
ts = timestamps[19700]
with timer('Full frame'):
    with timer('Generate frame'):
        fig = heatmap.animation_frame_fade(
            whales_segs, vessel_segs,
            whales_interp, vessel_points,
            protected_areas, basemap_src, bds, ts, vessel_encounters)
    with timer('Export frame'):
        show(fig)

In [ ]:
# Test static map
ranges = [('2020-07-01', '2023-06-01'),
          ('2020-07-01', '2021-06-01'),
          ('2021-07-01', '2022-06-01'),
          ('2022-07-01', '2023-06-01')]
range_labels = ['full', '2020', '2021', '2022']

timestamps = {
    label: pd.date_range(start, end, freq='30min')
    for label, (start, end) in zip(range_labels, ranges)
}

whale_mask = {
    label: (whales_interp.timestamp >= start) & (whales_interp.timestamp < end)
    for label, (start, end) in zip(range_labels, ranges)
}

vessel_mask = {
    label: (vessel_points.timestamp >= start) & (vessel_points.timestamp < end)
    for label, (start, end) in zip(range_labels, ranges)
}

encounters_mask = {
    label: (vessel_encounters.timestamp >= start) & (vessel_encounters.timestamp < end)
    for label, (start, end) in zip(range_labels, ranges)
}

bds = {
    'full': bounds_full,
    'auck': bounds_auck,
    'camp': bounds_camp,
    'anti': bounds_ant,
}

label = '2022'
bname = 'auck'
use_encounters = False

with timer('Full frame'):
    with timer('Generate frame'):
        fig = heatmap.animation_frame(
            whales_interp[whale_mask[label]], vessel_points[vessel_mask[label]],
            protected_areas, basemap_src, bds[bname],
            encounters=vessel_encounters[encounters_mask[label]] if use_encounters else None)
    with timer('Export frame'):
        show(fig)

In [ ]:
# Figure out how many frames in each yearly video
skip_no_whales = timestamps.map(lambda ts: any(whales_interp['timestamp'] == ts))
skipped_timestamps = timestamps[skip_no_whales]

timestamps = pd.date_range(start, end, freq='30min')
filtered_timestamps = pd.read_parquet(f'data/timestamps/filtered_timestamps_{start}_{end}_{3}.parquet').index


ranges = [('2020-07-01', '2023-06-01'),
          ('2020-07-01', '2021-06-01'),
          ('2021-07-01', '2022-06-01'),
          ('2022-07-01', '2023-06-01')]
range_labels = ['full', '2020', '2021', '2022']

timestamps_ranges = {
    label: pd.date_range(start, end, freq='30min')
    for label, (start, end) in zip(range_labels, ranges)
}

for label, (r_start, r_end) in zip(range_labels, ranges):
    num_full = len(timestamps[(timestamps >= r_start) & (timestamps < r_end)])
    num_filtered = len(filtered_timestamps[(filtered_timestamps >= r_start) & (filtered_timestamps < r_end)])
    num_skipped = len(skipped_timestamps[(skipped_timestamps >= r_start) & (skipped_timestamps < r_end)])
    
    print(f'{label} - #frames: {num_full} / {num_filtered} / {num_skipped} - time at 30fps: {num_full / 30 / 60:.2f} / {num_filtered / 30 / 60:.2f} / {num_skipped / 30 / 60:.2f} mins')
    # print(label, len(timestamps[(timestamps >= start) & (timestamps < end)]))
    # print(label, len(filtered_timestamps[(filtered_timestamps >= start) & (filtered_timestamps < end)]))

In [ ]:
# Test timestamp filtering with bounds
timestamps = pd.date_range(start, end, freq='30min')

whales_interp = gpd.read_parquet(f'data/intermediate/whale_pts_final_{start}_{end}.parquet')

has_whale = timestamps.map(lambda ts: any(whales_interp['timestamp'] == ts))

In [ ]:
auck_whales = whales_interp.cx[bounds_auck[0]:bounds_auck[2], bounds_auck[1]:bounds_auck[3]]

auck_has_whale = timestamps.map(lambda ts: any(auck_whales['timestamp'] == ts))

In [ ]:
print(len(whales_interp), sum(has_whale))
print(len(auck_whales), sum(auck_has_whale))